In [1]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

C:\Users\64800\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\64800\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\64800\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
idx2book = np.load('idx2book.npy', allow_pickle=True).item()
info_dict = np.load('info_dict.npy', allow_pickle=True).item()

In [3]:
embed_content = np.load('book_embeds_content.npy')
embed_collaborative = np.load('book_embeds_collaborative.npy')

In [48]:
# user_id: selected user,   int
# info_dict: information about books user already read,   {user_id: {book_id:rating}}
# embed: book embeddings,   (num_books, num_latent_features)
# topn_read: topn rated books that user already read,   int
# topn_similar: topn similar book to books that user already read,   int
# topn_recommend: topn recommendations to user,   int

def make_recommendation(user_id, info_dict, idx2book, embed_content, embed_collaborative, topn_read, topn_similar, topn_recommend):
    score_dict = {}

    user_reading = info_dict[user_id]
    user_read = {}
    for item in user_reading:
        user_read.update(item)
    topn_book_read = dict(sorted(user_read.items(), key=lambda x: x[1], reverse=True)[:topn_read])  # {book_id:rating}

    for book_id in topn_book_read:
        read_book_rating = topn_book_read[book_id]
        all_similarity = (cosine_similarity(embed_content[1].reshape(1,-1), embed_content).squeeze() + cosine_similarity(embed_collaborative[1].reshape(1,-1), embed_collaborative).squeeze()) / 2
        topn_similar_books = np.argsort(all_similarity)[::-1][1:topn_similar+1]
        topn_book_similarity = all_similarity[topn_similar_books]
        
        similarity_dict = {item[0]: item[1] for item in zip(topn_similar_books, topn_book_similarity)}
        for book in similarity_dict:
            candidate_book_similarity = similarity_dict[book]
            if book in score_dict:
                score_dict[book] += read_book_rating * candidate_book_similarity
            else:
                score_dict[book] = read_book_rating * candidate_book_similarity

    recommend_dict = dict(sorted(score_dict.items(), key=lambda x: x[1], reverse=True)[:topn_recommend])
    recommendation_idx = list(recommend_dict.keys())
    recommendation = [idx2book[idx] for idx in recommendation_idx]
    return recommendation, recommend_dict

In [49]:
# user_id: selected user,   int
# info_dict: information about books user already read,   {user_id: {book_id:rating}}
# embed: book embeddings,   (num_books, num_latent_features)
# topn_read: topn rated books that user already read,   int
# topn_similar: topn similar book to books that user already read,   int
# topn_recommend: topn recommendations to user,   int
user_id = 15
topn_read = 100
topn_similarity = 100
topn_recommend = 10
books, score_dict = make_recommendation(user_id, info_dict, idx2book, embed_content, embed_collaborative, topn_read, topn_similarity, topn_recommend)

In [50]:
books

['The Short Victorious War (Honor Harrington Series, Book 3)',
 'High Adventure in the Great Outdoors',
 'Not a Sparrow Falls',
 'Blood of the Fold (Sword of Truth, Book 3)',
 'I Am the Cheese (Laurel-Leaf Library)',
 'My Favorite Horror Story',
 'Among the Hidden',
 'All Around the World Cookbook',
 'Gift from the sea',
 "The Bad Girl's Guide to the Party Life"]

In [51]:
score_dict

{2628: 7.4992163479328156,
 18849: 7.498330771923065,
 29475: 7.498186826705933,
 25091: 7.498127818107605,
 46692: 7.498052716255188,
 7279: 7.498019188642502,
 12229: 7.497948110103607,
 18758: 7.497901618480682,
 20375: 7.497891783714294,
 8721: 7.497866302728653}